In [1]:
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from ltn_imp.automation.knowledge_base import KnowledgeBase
from torch.utils.data import Dataset, DataLoader

## Data Prep

In [2]:
def create_dataset(n_samples, vector_size):
    assert n_samples % 2 == 0, "n_samples must be even"
    
    # Generate random vectors
    instances = np.random.rand(n_samples, vector_size)
    
    # Initialize lists to store pairs and labels
    pairs = []
    labels = []
    
    # Create same pairs
    for i in range(n_samples // 2):
        idx = np.random.choice(n_samples)
        pair = (instances[idx], instances[idx])
        pairs.append(pair)
        labels.append(1)
    
    # Create different pairs
    for _ in range(n_samples // 2):
        idx1, idx2 = np.random.choice(n_samples, 2, replace=False)
        pair = (instances[idx1], instances[idx2])
        pairs.append(pair)
        labels.append(0)
    
    # Convert to numpy arrays
    pairs = np.array(pairs)
    labels = np.array(labels)
    
    return pairs, labels

In [3]:
class SameDataset(Dataset):
    def __init__(self, pairs, labels):
        self.pairs = pairs
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        x = self.pairs[idx][0]
        y = self.pairs[idx][1]
        label = self.labels[idx]
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

In [4]:
# Parameters
n_samples = 1000  # Must be even
vector_size = 10

# Generate the dataset
pairs, labels = create_dataset(n_samples, vector_size)

# Create the dataset
dataset = SameDataset(pairs, labels)

# Create the DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

## Model and Methods

In [5]:
import torch.nn as nn

class SameModel(nn.Module):
    def __init__(self, input_size):
        super(SameModel, self).__init__()
        self.fc1 = nn.Linear(input_size * 2, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
class LogitsToPredicate(torch.nn.Module):
    def __init__(self, logits_model):
        super(LogitsToPredicate, self).__init__()
        self.logits_model = logits_model
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, y, training=True):
        logits = self.logits_model(x)
        probs = self.softmax(logits)
        # y is expected to be a one-hot encoded vector
        out = torch.sum(probs * y, dim=1, keepdim=True)
        return out

In [6]:
def compute_accuracy(loader, model):
    mean_accuracy = 0.0
    for x,y, labels in loader:
        predictions = model.logits_model(x,y).detach().numpy()
        predictions = np.argmax(predictions, axis=1)
        mean_accuracy += accuracy_score(labels, predictions)
    return mean_accuracy / len(loader)

## Optimize 

In [7]:
a = SameModel(input_size=vector_size) # type: ignore
model = LogitsToPredicate(a) # type: ignore
predicates = {"Classifier": model}
expression_1 = "all x. (Classifier(x,y))"
rules = [expression_1]

In [8]:
rule_to_data_loader_mapping={expression_1: [dataloader]}

In [9]:
loader_to_variable = {dataloader: "x"}

In [10]:
loader_to_target = {dataloader: "y"}

In [11]:
loader_classes = { dataloader: 2}

In [12]:
kb = KnowledgeBase(rules=rules, predicates=predicates,
                   rule_to_data_loader_mapping=rule_to_data_loader_mapping,
                   loader_classes=loader_classes, # type: ignore
                   loader_to_variable=loader_to_variable, # type: ignore
                   loader_to_target=loader_to_target, # type: ignore 
                   quantifier_impls={"forall" : "pmean_error"})

In [13]:
compute_accuracy(model = model, loader= dataloader)

0.5029296875

In [14]:
kb.optimize()

ValueError: too many values to unpack (expected 2)

In [15]:
compute_accuracy(model = model, loader= dataloader)

0.5029296875